In [1]:
import torch 
import torch.nn
from torch import nn
import numpy as np

In [2]:
from PIL import Image
def get_img():
    im = Image.open('testim_1.png')

    im_array = np.vstack((np.expand_dims(np.asarray(im),0),np.expand_dims(np.asarray(im),0)))
    im_array.shape
    im_array = np.transpose(im_array,(0,3,1,2))
    im_array= im_array[:,:,0:49]
    im_array.shape
    im_array = torch.tensor(im_array)[:,0:3].to(torch.float32)
    
    return im_array

im_array = get_img()


In [3]:
im_array.shape

torch.Size([2, 3, 49, 2910])

## Critic shapes

In [4]:
pool = nn.MaxPool2d(2)
colorchs = 3 
dims=[8, 8, 8, 16]
stride = 1
bottleneck=32
chfak=1 

print(im_array.shape)
im_array = nn.Conv2d(colorchs, dims[0], 3, stride, 1)(im_array)
print(im_array.shape)
im_array = pool(im_array)
print(im_array.shape)
im_array = nn.Conv2d(dims[0], dims[1], 3, stride, 1)(im_array)
print(im_array.shape)
im_array =pool((im_array))
print(im_array.shape)
im_array = nn.Conv2d(dims[1], dims[2], 3, stride, 1)(im_array)
print(im_array.shape)
im_array = pool(im_array)
print(im_array.shape)
im_array = nn.Conv2d(dims[2], dims[3], 3, stride, 1)(im_array)
print(im_array.shape)
#im_array = pool(im_array)
#print(im_array.shape)
im_array  = nn.Conv2d(dims[3], bottleneck * chfak, 4)(im_array)
print(im_array.shape)
im_array  = nn.Flatten()(im_array)
print(im_array.shape)
im_array = nn.Linear(480, chfak * bottleneck)(im_array)
print(im_array.shape)
im_array = nn.Linear(chfak * bottleneck, 1)(im_array)
print(im_array.shape)

torch.Size([2, 3, 49, 2910])
torch.Size([2, 8, 49, 2910])
torch.Size([2, 8, 24, 1455])
torch.Size([2, 8, 24, 1455])
torch.Size([2, 8, 12, 727])
torch.Size([2, 8, 12, 727])
torch.Size([2, 8, 6, 363])
torch.Size([2, 16, 6, 363])
torch.Size([2, 32, 3, 360])
torch.Size([2, 34560])


RuntimeError: mat1 and mat2 shapes cannot be multiplied (2x34560 and 480x32)

In [ ]:
dims=[32, 64, 128, 256]

In [5]:
X = get_img()
ch = 3 
k=3
step = 1
p =  1
CRAFTER_BOTTLENECK = 13312
latent_dim = 64


# encoder

print(X.shape)
X = nn.Conv2d(ch, dims[0], k, step, p)(X)
print(X.shape)
X = nn.Conv2d(dims[0], dims[0], 3, 2, 1)(X)
print(X.shape)
X = nn.Conv2d(dims[0], dims[1], k, step, p)(X)
print(X.shape)
X = nn.Conv2d(dims[1], dims[1], 3, 2, 1)(X)
print(X.shape)


mu = nn.Linear(CRAFTER_BOTTLENECK, latent_dim)(nn.Flatten()(X))
logvar = nn.Linear(CRAFTER_BOTTLENECK, latent_dim)(nn.Flatten()(X))

print('\tmu: ',mu.shape,'var: ',logvar.shape)



# decoder

# reparameterization same shape as mu itself
X = torch.zeros(2,latent_dim+1)
print(X.shape)
X = nn.Linear(latent_dim+1,CRAFTER_BOTTLENECK)(X)
print(X.shape)
X = X.view(-1,64,13,16)
print(X.shape)
X = torch.nn.ConvTranspose2d(in_channels=64, out_channels=32, kernel_size=3, stride=(2,3), padding=(1,8), dilation=1)(X)
print(X.shape)
X =  torch.nn.ConvTranspose2d(in_channels=32, out_channels=32, kernel_size=3, stride=(2,3), padding=(1,16), dilation=1)(X)
print(X.shape)
X =   nn.Conv2d(dims[0], ch, k, step, p) (X)
print(X.shape)

torch.Size([2, 3, 49, 2910])
torch.Size([2, 8, 49, 2910])
torch.Size([2, 8, 25, 1455])
torch.Size([2, 8, 25, 1455])
torch.Size([2, 8, 13, 728])


RuntimeError: mat1 and mat2 shapes cannot be multiplied (2x75712 and 13312x64)

In [ ]:

print(X.shape)

k = 3 
ch = 3
step = 1
p= 1
dims=[32, 64, 128, 256]

In [ ]:
X = get_img()

X=          nn.Conv2d(ch, dims[0], k, step, p)(X) 
print(X.shape)
X=          nn.BatchNorm2d(dims[0])(X)
print(X.shape)
X=          nn.MaxPool2d(2)(X)
print(X.shape)

X=            nn.Conv2d(dims[0], dims[1], k, step, p)(X)
print(X.shape)
X=            nn.BatchNorm2d(dims[1])(X)
print(X.shape)
X=            nn.MaxPool2d(2)(X)
print(X.shape)


X=            nn.Conv2d(dims[1], dims[2], k, step, p)(X)
print(X.shape)
X=           nn.BatchNorm2d(dims[2])(X)
print(X.shape)
#X=            nn.MaxPool2d(2)(X)
print(X.shape)


X=            nn.Conv2d(dims[2], dims[3], k, step, p)(X)
print(X.shape)
X=            nn.BatchNorm2d(dims[3])(X)
print(X.shape)
#X=            nn.MaxPool2d(2)(X)
print(X.shape)


In [ ]:
X=  nn.Conv2d(dims[3], dims[2], k, step, p)(X)
print(X.shape)
X=             nn.Upsample(scale_factor=2)(X)
print(X.shape)
X=             nn.Conv2d(dims[2], dims[1], k, step, p)(X)
print(X.shape)
X=             nn.Upsample(scale_factor=2)(X)
print(X.shape)
X=             nn.Conv2d(dims[1], dims[0], k, step, p)(X)
print(X.shape)
#X=             nn.Upsample(scale_factor=2)(X)
print(X.shape)
X=             nn.Conv2d(dims[0], dims[0], k, step, p)(X)
print(X.shape)
#X=             nn.Upsample(scale_factor=2)(X)
print(X.shape)
X=             nn.Conv2d(dims[0], ch, k, step, p)(X)
print(X.shape)

In [ ]:
X.shape

In [ ]:
import pandas as pd

pd.read_csv('/home/olli/gits/Critic-VAE/log.csv')['total_loss'][:].plot()

In [ ]:
pov = np.load('/home/olli/gits/Critic-VAE/pov.np.npy')
recon = np.load('/home/olli/gits/Critic-VAE/revon.np.npy')


In [ ]:
im_array = get_img().numpy()[0]
im_array = im_array.squeeze().transpose(1,2,0)
im_array.shape

In [ ]:
def plot_side_by_side(filename,ims, labels = None):
    import matplotlib.pyplot as plt


    f, axs = plt.subplots(1,len(ims))
    if labels == None:
        labels = range(len(ims))
    if len(labels) < len(ims):
        labels.extend(['none']*(len(ims)-len(labels)))
    
    for ax, im, label  in zip(axs,ims,labels[0:len(ims)]):
        im = im.squeeze().transpose(1,2,0)*255
        ax.imshow(im.astype(np.uint8))
        ax.title.set_text(str(label))
        ax.axis('off')
    f.suptitle('test 12312321 31232123 13 21123 23 1',y=0.7)
    plt.savefig(filename,bbox_inches='tight')
    
plot_side_by_side('test2.jpg',[pov,recon,recon,recon,pov],['train',1,2,3,4,5,6])

In [ ]:
[1,2,3,4,5][0:2
           ]

In [ ]:
x = torch.ones((3, 48, 64))
y = torch.ones((3, 48, 64))
torch.stack([x,y]).shape